# Secuencial

In [4]:
import time
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
def cargar_archivo_fna(nombre_archivo):
    with open(nombre_archivo, 'r') as archivo:
        contenido = archivo.read()
    return contenido

In [5]:
secuencia1 = cargar_archivo_fna("data/E_coli.fna")
secuencia2 = cargar_archivo_fna("data/Salmonella.fna")

In [11]:
begin = time.time()
dotplot = np.empty([len(secuencia1),len(secuencia2)], dtype=np.bool_)
print("La matriz de resultado tiene tamaño: ", dotplot.shape)

dotplot[secuencia1[:, None] == secuencia2] = True

print(f"\n El código se ejecutó en: {time.time() - begin} segundos")

MemoryError: Unable to allocate 21.4 TiB for an array with shape (4699745, 5013482) and data type bool

# el codigo completo saldria asi

Aca se esta utilizando una matriz dispersa: en lugar de crear una matriz completa, puede utilizar matriz dispersa para almacenar solo las posiciones y valores no nulos.

Tambien se va a procesar por lotes, se divide el procesamiento en lotes mas pequeños en lugar de cargar todo el archivo a la vez

In [ ]:
import time
import numpy as np
from scipy.sparse import lil_matrix
from tqdm import tqdm

def cargar_archivo_fna(nombre_archivo):
    with open(nombre_archivo, 'r') as archivo:
        contenido = archivo.read()
    return contenido

def crear_dotplot(secuencia1, secuencia2):
    codigos_secuencia1 = np.array([ord(c) for c in secuencia1], dtype=np.uint8)
    codigos_secuencia2 = np.array([ord(c) for c in secuencia2], dtype=np.uint8)

    dotplot = lil_matrix((len(secuencia1), len(secuencia2)), dtype=np.uint8)
    for i in tqdm(range(len(secuencia1))):
        for j in range(len(secuencia2)):
            if codigos_secuencia1[i] == codigos_secuencia2[j]:
                dotplot[i, j] = 1
            else:
                dotplot[i,j] = 0
    return dotplot

secuencia1 = cargar_archivo_fna("E_coli.fna")
secuencia2 = cargar_archivo_fna("Salmonella.fna")

begin = time.time()
dotplot = crear_dotplot(secuencia1, secuencia2)
print("La matriz de resultado tiene tamaño: ", dotplot.shape)
print(f"\n El código se ejecutó en: {time.time() - begin} segundos")

In [ ]:
import time
import numpy as np
from scipy.sparse import lil_matrix
from tqdm import tqdm

def cargar_archivo_fna(nombre_archivo):
    with open(nombre_archivo, 'r') as archivo:
        contenido = archivo.read()
    return contenido

secuencia1 = cargar_archivo_fna("E_coli.fna")
secuencia2 = cargar_archivo_fna("Salmonella.fna")
bloque_tam = 10000  # Tamaño del bloque a procesar

begin = time.time()

dotplot = lil_matrix((len(secuencia1), len(secuencia2)), dtype=np.uint8)
print("La matriz de resultado tiene tamaño: ", dotplot.shape)

for i in tqdm(range(0, len(secuencia1), bloque_tam)):
    bloque_secuencia1 = secuencia1[i:i+bloque_tam]
    for j in range(0, len(secuencia2), bloque_tam):
        bloque_secuencia2 = secuencia2[j:j+bloque_tam]
        for k in range(len(bloque_secuencia1)):
            for l in range(len(bloque_secuencia2)):
                if bloque_secuencia1[k] == bloque_secuencia2[l]:
                    dotplot[i+k, j+l] = 1
                else:
                    dotplot[i+k, j+l] = 0
    


print(f"\n El código se ejecutó en: {time.time() - begin} segundos")

In [ ]:
import matplotlib.pyplot as plt
def draw_dotplot(matrix, fig_name='dotplot.svg'):
  plt.figure(figsize=(5,5))
  plt.imshow(matrix, cmap='Greys',aspect='auto')

  plt.ylabel("Secuencia 1")
  plt.xlabel("Secuencia 2")
  plt.savefig(fig_name)

In [ ]:
draw_dotplot(dotplot[:200,:200 ])

# Multiprocessing

In [ ]:
%%writefile multiprocessing.py
import argparse
from Bio import SeqIO
import matplotlib.pyplot as plt
import numpy as np
from scipy.sparse import lil_matrix

# Función para generar el dotplot utilizando una matriz dispersa
def sparse_dotplot(seq1, seq2):
    n = len(seq1)
    m = len(seq2)
    dotplot = lil_matrix((n, m), dtype=bool)

    for i in range(n):
        for j in range(m):
            if seq1[i] == seq2[j]:
                dotplot[i, j] = True

    return dotplot

# Función principal para ejecutar el dotplot
def run_dotplot(seq1_file, seq2_file):
    # Leer las secuencias desde los archivos FASTA
    seq1 = str(next(SeqIO.parse(seq1_file, "fasta")).seq)
    seq2 = str(next(SeqIO.parse(seq2_file, "fasta")).seq)

    dotplot = sparse_dotplot(seq1, seq2)

    # Generar el gráfico del dotplot
    plt.imshow(dotplot.toarray(), cmap="Greys")
    plt.title("Dotplot")
    plt.xlabel("Sequence 2")
    plt.ylabel("Sequence 1")
    plt.show()

if _name_ == "_main_":
    # Configurar los argumentos de línea de comandos
    parser = argparse.ArgumentParser(description="Dotplot Generator")
    parser.add_argument("seq1_file", type=str, help="Archivo FASTA de la secuencia 1")
    parser.add_argument("seq2_file", type=str, help="Archivo FASTA de la secuencia 2")
    args = parser.parse_args()

    # Ejecutar el dotplot con los argumentos proporcionados
    run_dotplot(args.seq1_file, args.seq2_file)

In [ ]:
!python3 multiprocessing.py E_coli.fna Salmonella.fna

# MPI4PY

In [9]:
%%writefile dotplot_mpi.py
import argparse
from Bio import SeqIO
import matplotlib.pyplot as plt
import numpy as np
from mpi4py import MPI

# Función para generar el dotplot utilizando mpi4py
def distributed_dotplot(seq1, seq2):
    comm = MPI.COMM_WORLD
    rank = comm.Get_rank()
    size = comm.Get_size()

    n = len(seq1)
    m = len(seq2)
    dotplot = np.zeros((n, m))

    # Dividir las filas de trabajo entre los procesos
    chunk_size = n // size
    start_row = rank * chunk_size
    end_row = start_row + chunk_size if rank < size - 1 else n

    for i in range(start_row, end_row):
        for j in range(m):
            if seq1[i] == seq2[j]:
                dotplot[i, j] = 1

    # Recopilar los resultados parciales en el proceso raíz
    dotplot = comm.gather(dotplot, root=0)

    if rank == 0:
        dotplot = np.concatenate(dotplot)

    return dotplot

# Función principal para ejecutar el dotplot
def run_dotplot(seq1_file, seq2_file):
    # Leer las secuencias desde los archivos FASTA
    seq1 = str(next(SeqIO.parse(seq1_file, "fasta")).seq)
    seq2 = str(next(SeqIO.parse(seq2_file, "fasta")).seq)

    dotplot = distributed_dotplot(seq1, seq2)

    # Generar el gráfico del dotplot
    plt.imshow(dotplot, cmap="Greys")
    plt.title("Dotplot")
    plt.xlabel("Sequence 2")
    plt.ylabel("Sequence 1")
    plt.show()

if __name__ == "__main__":
    # Configurar los argumentos de línea de comandos
    parser = argparse.ArgumentParser(description="Dotplot Generator")
    parser.add_argument("seq1_file", type=str, help="Archivo FASTA de la secuencia 1")
    parser.add_argument("seq2_file", type=str, help="Archivo FASTA de la secuencia 2")
    args = parser.parse_args()

    # Ejecutar el dotplot con los argumentos proporcionados
    run_dotplot(args.seq1_file, args.seq2_file)

Writing dotplot_mpi.py


In [11]:
!mpiexec -n 6 python3 dotplot_mpi.py E_coli.fna Salmonella.fna

Traceback (most recent call last):
  File "/home/mrtopo/Documents/Proyects/Analisis-Rendimiento-Dotplot/dotplot_mpi.py", line 58, in <module>
    run_dotplot(args.seq1_file, args.seq2_file)
  File "/home/mrtopo/Documents/Proyects/Analisis-Rendimiento-Dotplot/dotplot_mpi.py", line 41, in run_dotplot
    dotplot = distributed_dotplot(seq1, seq2)
  File "/home/mrtopo/Documents/Proyects/Analisis-Rendimiento-Dotplot/dotplot_mpi.py", line 15, in distributed_dotplot
    dotplot = np.zeros((n, m))
numpy.core._exceptions._ArrayMemoryError: Unable to allocate 164. TiB for an array with shape (4641652, 4857450) and data type float64
Traceback (most recent call last):
  File "/home/mrtopo/Documents/Proyects/Analisis-Rendimiento-Dotplot/dotplot_mpi.py", line 58, in <module>
    run_dotplot(args.seq1_file, args.seq2_file)
  File "/home/mrtopo/Documents/Proyects/Analisis-Rendimiento-Dotplot/dotplot_mpi.py", line 41, in run_dotplot
    dotplot = distributed_dotplot(seq1, seq2)
  File "/home/mrtopo/Doc